In [2]:
import requests
import csv
import json
from bs4 import BeautifulSoup
"""Trying to get a whole list of phrasal verbs"""

'Trying to get a whole list of phrasal verbs'

In [23]:
def getList():
    urlReq = requests.get("https://www.usingenglish.com/reference/phrasal-verbs/list.html")
    soup = BeautifulSoup(urlReq.content, "lxml")
    anchorList =soup.select(".pv-searchres ul li a")
    pvList = [["Phrasal Verb","Link"]]
    for anchor in anchorList:
        pvList.append([anchor.text,"https://www.usingenglish.com"+anchor["href"]])

    with open("Phrasal Verbs.csv","w+") as pvfile:
        csv.writer(pvfile).writerows(pvList)

    print(len(pvList))
    
getList()

2377

In [8]:
def testOnePage():
    req = requests.get("https://www.usingenglish.com/reference/phrasal-verbs/abide+by.html")

    def clean(p):
        p.strong.clear()
        return p.text.strip()
    sopa = BeautifulSoup(req.content,"lxml")
    info = sopa.find("div", class_="indent")

    meaning,example,_ = info.find_all("p")
    separable = info.find("li").text.strip()
    meaning = clean(meaning)
    example = clean(example)
    print("{} - {} - {}".format(meaning,example,separable))

testOnePage()

Accept or follow a decision or rule - We have to ABIDE BY what the court says. - Inseparable


In [9]:
allInfo = [] # Not to be global, but is useful for reusing it in Jupyter
def getEachPhrasal():
    """Time to scrape each URL looking for definitions"""
    global allInfo

    def clean(p):
        p.strong.clear()
        return p.text.strip()

    with open("Phrasal Verbs.csv") as linkList:
        reader = csv.DictReader(linkList)
        
        for i,row in enumerate(reader):
            html = requests.get(row["Link"])
            soup = BeautifulSoup(html.content,"lxml")
            info = soup.find("div", class_="indent")
            meaning,example,_ = info.find_all("p")
            row["Meaning"] = clean(meaning)
            row["Example"] = clean(example)
            notes = info.find_all("li")
            row["Notes"] = []
            for note in notes:
                row["Notes"].append(note.text.strip())
            allInfo.append(row.copy())
            if not i % 50 and i:
                json.dump(allInfo,open("allPhrasal.json","w+"))
                print("Saved",i,"phrasal verbs into json")
    json.dump(allInfo,open("allPhrasal.json","w+"))
getEachPhrasal()

Saved 50 phrasal verbs into json
Saved 100 phrasal verbs into json
Saved 150 phrasal verbs into json
Saved 200 phrasal verbs into json
Saved 250 phrasal verbs into json
Saved 300 phrasal verbs into json
Saved 350 phrasal verbs into json
Saved 400 phrasal verbs into json
Saved 450 phrasal verbs into json
Saved 500 phrasal verbs into json
Saved 550 phrasal verbs into json
Saved 600 phrasal verbs into json
Saved 650 phrasal verbs into json
Saved 700 phrasal verbs into json
Saved 750 phrasal verbs into json
Saved 800 phrasal verbs into json
Saved 850 phrasal verbs into json
Saved 900 phrasal verbs into json
Saved 950 phrasal verbs into json
Saved 1000 phrasal verbs into json
Saved 1050 phrasal verbs into json
Saved 1100 phrasal verbs into json
Saved 1150 phrasal verbs into json
Saved 1200 phrasal verbs into json
Saved 1250 phrasal verbs into json
Saved 1300 phrasal verbs into json
Saved 1350 phrasal verbs into json
Saved 1400 phrasal verbs into json
Saved 1450 phrasal verbs into json
Saved

In [15]:
def showMeKeys():
    """We are not sure how many notes are in the list"""
    maxNotes = 0
    minNotes = 2
    allDataScrapped = json.load(open("allPhrasal.json"))
    for i,dictionary in enumerate(allDataScrapped):
        maxNotes = max(maxNotes,len(dictionary["Notes"]))
        minNotes = min(minNotes,len(dictionary["Notes"]))
    print("Max notes are:",maxNotes,"Min notes:",minNotes)
    keys = list(allDataScrapped[0].keys())
    keys.reverse()
    keys.remove("Notes")
    keys += ["Note "+str(x+1) for x in range(maxNotes)]
    print(keys)
showMeKeys()

Max notes are: 3 Min notes: 1
['Meaning', 'Link', 'Example', 'Phrasal Verb', 'Note 1', 'Note 2', 'Note 3']


In [28]:
def jsonToCsv():
    from collections import defaultdict
    
    allDataScrapped = json.load(open("allPhrasal.json"))
    for index,dictionary in enumerate(allDataScrapped):
        rowDict = defaultdict(str)
        for key,value in dictionary.items():
            if key != "Notes":
                rowDict[key] = value
            else:
                if len(value) == 1:
                    rowDict["Note 2"] = note
                else:
                    for order,note in enumerate(value):
                            rowDict["Note "+str(order+1)] = note
        del allDataScrapped[index]["Notes"]
        allDataScrapped[index] = rowDict            
        
    
    keys = ['Phrasal Verb', 'Meaning', 'Example', 'Note 1', 'Note 2', 'Note 3', 'Link']
    with open("Phrasal Verbs Definitions.csv", "w+") as pointFile:
        csv.DictWriter(pointFile, fieldnames=keys).writeheader()
    with open("Phrasal Verbs Definitions.csv", "a") as pointFile:
        writer = csv.DictWriter(pointFile, fieldnames=keys)
        writer.writerows(allDataScrapped)
jsonToCsv()